## Working with PDFs with images

PDFs contain more than rich formatting - they have images!

Run the cells below to convert PDF data to images (this should take about a minute). 

In [1]:
try:
    import pymupdf
except ImportError:
    %pip install -Uqq pymupdf

In [2]:
%%bash
python pdf_to_img.py
echo "Images extracted from PDFs"

Converting data/pdfs/amazon-2025-08-8k-excerpts.pdf to images...
Converted 12 pages to images
Converting data/pdfs/hai_ai-index-report-2025_chapter2_excerpts.pdf to images...
Converted 8 pages to images
Converting data/pdfs/reddit_s1a.pdf to images...
Converted 288 pages to images
Images extracted from PDFs


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

images = [
    f"data/imgs/hai_ai-index-report-2025_chapter2_excerpts_1_of_8.jpg",
    f"data/imgs/hai_ai-index-report-2025_chapter2_excerpts_4_of_8.jpg",
    f"data/imgs/hai_ai-index-report-2025_chapter2_excerpts_5_of_8.jpg"   
]

fig, axes = plt.subplots(1, 3, figsize=(15, 40))

for i, img_path in enumerate(images):
    img = Image.open(img_path)
    axes[i].imshow(img)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

How do we work with these for RAG?

### Approach 1 - Extract text and images separately

Some libraries (like `docling`) can extract text and images from PDFs, and convert them into Markdown files.

Here, we've pre-converted this PDF into markdown.

In [ ]:
md_filepath = Path("data/parsed/hai_ai-index-report-2025_chapter2_excerpts-parsed-w-imgs.md")
md_txt = md_filepath.read_text()
print(md_txt[:1000])

#### Chunking text files with images

More complex than just text, since we need to handle images as well.

- Must include entire image string in the chunk
- When vectorizing, optionally include base64 of image
    - Your embedding model must be multimodal

Chunking becomes more complex.

One method: try a specialized library like `chonkie` to handle this

Chonkie offers a variety of chunking strategies:

<img src="images/chonkie_methods.png" />

There isn't going to be a "one size fits all" solution for chunking PDFs with images. But these libraries can help you get started.

Let's try a couple of different approaches:

In [ ]:
from chonkie import RecursiveChunker

# Initialize the recursive chunker to chunk Markdown
chunker = RecursiveChunker.from_recipe("markdown", lang="en")

In [ ]:
chunk_texts = chunker.chunk(md_txt)

In [ ]:
import textwrap

for chunk in chunk_texts[:5]:
    print(f"\n" + "=" * 40)
    print(f"Token count: {chunk.token_count}")
    print(f"Chunk text:")
    wrapped_text = textwrap.fill(chunk.text[:500]+"...", width=80)
    print(textwrap.indent(wrapped_text, "    "))

Let's try a "semantic" chunker:

In [ ]:
from chonkie import SemanticChunker

# Basic initialization with default parameters
chunker = SemanticChunker(
    embedding_model="minishlab/potion-base-8M",  # Default model
    threshold=0.5,                               # Similarity threshold (0-1) or (1-100) or "auto"
    chunk_size=2048,                              # Maximum tokens per chunk
    min_sentences=1                              # Initial sentences per chunk
)

In [ ]:
# Chunk text into `chunk_texts` as we've done before
# BEGIN_SOLUTION
chunk_texts = chunker.chunk(md_txt)
# END_SOLUTION

In [ ]:
for chunk in chunk_texts[:5]:
    print(f"\n" + "=" * 40)
    print(f"Token count: {chunk.token_count}")
    print(f"Chunk text:")
    wrapped_text = textwrap.fill(chunk.text[:500]+"...", width=80)
    print(textwrap.indent(wrapped_text, "    "))

### Set up Weaviate Collection

In [ ]:
import weaviate
import os

client = weaviate.connect_to_embedded(
    version="1.32.0",
    headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),
    },
    environment_variables={"LOG_LEVEL": "error"}  # Reduce amount of logs
)

In [ ]:
client.collections.delete("Chunks")

In [ ]:
from weaviate.classes.config import Property, DataType, Configure, Tokenization

client.collections.create(
    name="Chunks",
    properties=[
        Property(
            name="document_title",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk_number",
            data_type=DataType.INT,
        ),
        Property(
            name="filename",
            data_type=DataType.TEXT,
            tokenization=Tokenization.FIELD
        ),
    ],
    vector_config=[
        Configure.Vectors.text2vec_cohere(
            name="default",
            source_properties=["document_title", "chunk"],
            model="embed-v4.0"
        )
    ]
)

In [ ]:
chunks = client.collections.use("Chunks")

### Import data

In [ ]:
from tqdm import tqdm

with chunks.batch.fixed_size(batch_size=100) as batch:
    for i, chunk_text in tqdm(enumerate(chunk_texts)):
        obj = {
            "document_title": "HAI AI Index Report 2025",
            "filename": "data/pdfs/hai_ai-index-report-2025_chapter2_excerpts.pdf",
            "chunk": chunk_text.text,
            "chunk_number": i + 1,
        }

        # Add object to batch for import with (batch.add_object())
        # BEGIN_SOLUTION
        batch.add_object(
            properties=obj
        )
        # END_SOLUTION

### RAG queries

How do we perform RAG in this scenario? 

This is a bit different, because we haven't embedded the images (or stored them in Weaviate).

In this scenario, let's:

- Retrieve text chunks
- Get images referred to in the text
- Convert the images to base64
- Send (retrieved text + images + prompt) to LLM for RAG

In [ ]:
response = chunks.query.hybrid(
    query="Latest in self-driving cars",
    limit=10
)

for o in response.objects:
    print(f"\n" + "=" * 40)
    print(o.properties["chunk"][:1000] + "...")

In [ ]:
import re

def extract_image_paths(text):
    """Extract image paths from markdown-style image references."""
    pattern = r'!\[.*?\]\((.*?)\)'
    return re.findall(pattern, text)

In [ ]:
def get_image_base64s(image_paths, base_path=None):
    import base64
    base64_images = []
    for img_path in image_paths:
        full_path = Path(base_path) / img_path if base_path else Path(img_path)
        image_bytes = full_path.read_bytes()
        base64_string = base64.b64encode(image_bytes).decode("utf-8")
        base64_images.append(base64_string)

    return base64_images

In [ ]:
all_chunks = ""
all_images = []

for o in response.objects:
    chunk_text = o.properties["chunk"]
    image_paths = extract_image_paths(chunk_text)
    all_images.extend(get_image_base64s(image_paths, base_path="data/parsed"))

    all_chunks += "\n\n" + chunk_text

In [ ]:
task_text = """
What does this tell us about the latest in self-driving cars

Describe the details from the figures as well, if necessary.
""" + "\n\n" + all_chunks

message = {
    "role": "user", "content": [
        {"type": "text", "text": task_text}
    ]
}

for img in all_images:
    content = {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": "image/png",
            "data": img,
        }
    }
    # Append `content`` to message["content"]
    # BEGIN_SOLUTION
    message["content"].append(content)
    # END_SOLUTION

In [ ]:
import anthropic

anthropic_response = anthropic.Anthropic().messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    # Add [message] as the messages to pass to Claude
    # BEGIN_SOLUTION
    messages=[message]
    # END_SOLUTION
)

In [ ]:
print(anthropic_response.content[0].text)

In [ ]:
client.close()